#### Install system packages for ODBC

In [2]:
!apt-get update
!apt-get install gcc
!apt-get install -y tdsodbc unixodbc-dev
!apt install unixodbc-bin -y
!apt-get clean 

Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:5 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [24.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2221 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2658 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1418 kB]
Get:13 http://archive.ubu

Get:7 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxext6 amd64 2:1.3.3-1 [29.4 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/main amd64 fontconfig amd64 2.12.6-0ubuntu2 [169 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmng2 amd64 2.0.2-0ubuntu3 [169 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 x11-common all 1:7.7+19ubuntu7.1 [22.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/main amd64 libice6 amd64 2:1.0.9-2 [40.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsm6 amd64 2:1.2.2-1 [15.8 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxt6 amd64 1:1.1.5-1 [160 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaudio2 amd64 1.9.4-6 [50.3 kB]
Get:15 http://archive.ubuntu.com/ubuntu bionic/main amd64 mysql-common all 5.8+1.0.4 [7308 B]
Get:16 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmysqlclient20 amd64 5.7.34-0ubuntu0.18.04.1 [689 kB]
Get:17 http://archiv

7Progress: [ 22%] [############..............................................] 8Selecting previously unselected package mysql-common.
Preparing to unpack .../14-mysql-common_5.8+1.0.4_all.deb ...
7Progress: [ 23%] [#############.............................................] 8Unpacking mysql-common (5.8+1.0.4) ...
7Progress: [ 24%] [##############............................................] 8Selecting previously unselected package libmysqlclient20:amd64.
Preparing to unpack .../15-libmysqlclient20_5.7.34-0ubuntu0.18.04.1_amd64.deb ...
Unpacking libmysqlclient20:amd64 (5.7.34-0ubuntu0.18.04.1) ...
7Progress: [ 25%] [##############............................................] 8Selecting previously unselected package qtcore4-l10n.
Preparing to unpack .../16-qtcore4-l10n_4%3a4.8.7+dfsg-7ubuntu1_all.deb ...
7Progress: [ 26%] [###############...........................................] 8Unpacking qtcore4-l10n (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 27%] [###############..........

7Progress: [ 59%] [##################################........................] 8Setting up libjbig0:amd64 (2.1-3.1build1) ...
7Progress: [ 60%] [##################################........................] 8Setting up qtcore4-l10n (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 61%] [###################################.......................] 8Setting up mysql-common (5.8+1.0.4) ...
7Progress: [ 62%] [####################################......................] 8update-alternatives: using /etc/mysql/my.cnf.fallback to provide /etc/mysql/my.cnf (my.cnf) in auto mode
Setting up qtchooser (64-ga1b6736-5) ...
7Progress: [ 63%] [####################################......................] 8Setting up libtiff5:amd64 (4.0.9-5ubuntu0.4) ...
7Progress: [ 65%] [#####################################.....................] 87Progress: [ 65%] [#####################################.....................] 8Setting up libmng2:amd64 (2.0.2-0ubuntu3) ...
7Progress: [ 66%] [###########################

#### Use this command to troubleshoot a failed pyodbc installation:
!pip install --upgrade --global-option=build_ext --global-option="-I/usr/local/include"  --global-option="-L/usr/local/lib" pyodbc

In [3]:
!pip install pyodbc
!pip install csv

  Using cached pyodbc-4.0.31.tar.gz (280 kB)
  Created wheel for pyodbc: filename=pyodbc-4.0.31-cp36-cp36m-linux_x86_64.whl size=280102 sha256=9711c2cee04e5d4cd6c7b600c30b1ffb69415b027bdf4964d3f19d75dc81b8ee
  Stored in directory: /root/.cache/pip/wheels/9f/f8/a0/5861e87f5e45d0f705d8d39d3ee1e480147ca59f784250c5c1
Successfully built pyodbc


In [4]:
!rm /etc/odbcinst.ini
!rm /etc/odbc.ini

In [5]:
!ln -s /tf/odbcinst.ini /etc/odbcinst.ini
!ln -s /tf/odbc.ini /etc/odbc.ini

In [6]:
!cat /tf/odbcinst.ini

[InterSystems ODBC35]
UsageCount=1
Driver=/tf/libirisodbcu35.so
Setup=/tf/libirisodbcu35.so
SQLLevel=1
FileUsage=0
DriverODBCVer=02.10
ConnectFunctions=YYN
APILevel=1
DEBUG=1
CPTimeout=<not pooled>



In [7]:
!cat /tf/odbc.ini

[user]
Driver=InterSystems ODBC35
Protocol=TCP
Host=irisimlsvr
Port=51773
Namespace=USER
UID=SUPERUSER
Password=SYS
Description=Sample namespace
Query Timeout=0
Static Cursors=0



In [8]:
!odbcinst -j

unixODBC 2.3.4
DRIVERS............: /etc/odbcinst.ini
SYSTEM DATA SOURCES: /etc/odbc.ini
FILE DATA SOURCES..: /etc/ODBCDataSources
USER DATA SOURCES..: /root/.odbc.ini
SQLULEN Size.......: 8
SQLLEN Size........: 8
SQLSETPOSIROW Size.: 8


#### Get an ODBC connection 

In [ ]:
import pyodbc 
import time

dsn = 'IRIS QuickML demo via PyODBC'
server = 'irisimlsvr' #'192.168.99.101' 
port = '51773' #'9091'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')
cursor = cnxn.cursor()

####  Prepare training data, and give a model name

In [18]:
dataTable = 'DATA.BitcoinRates'
dataTablePredict = 'BitcoinRatesResult'
dataColumn =  'Rate'
dataColumnPredict = "PredictedRate"
modelName = "SVR" #chose a name - must be unique in server end

#### Train model and predict rate

In [16]:
cursor.execute("CREATE MODEL %s PREDICTING (%s)  FROM %s" % (modelName, dataColumn, dataTable))
cursor.execute("TRAIN MODEL %s FROM %s" % (modelName, dataTable))
cursor.execute("Create Table %s (%s VARCHAR(100), %s VARCHAR(100))" % (dataTablePredict, dataColumnPredict, dataColumn))
cursor.execute("INSERT INTO %s  SELECT TOP 20 PREDICT(%s) AS %s, %s FROM %s" % (dataTablePredict, modelName, dataColumnPredict, dataColumn, dataTable)) 
cnxn.commit()

#### Look at result

In [ ]:
import pandas as pd
from IPython.display import display

df1 = pd.read_sql("SELECT * from %s ORDER BY ID" % dataTablePredict, cnxn)
display(df1)

In [ ]:
cnxn.close()